In [1]:
import numpy as np
import pandas as pd
import glob
import xarray as xr
import netCDF4 
from tqdm import tqdm_notebook as tqdm
import os
from datetime import date
import datetime

In [2]:
#Load BAHAMAS files (version from DLR-HALO-DATABASE)
file_list = sorted(glob.glob("/Users/marcusklingebiel/Desktop/BAHAMAS/"+"*.nc"))

prefix = "EUREC4A_10Hz_Position_Attitude_"

In [3]:
file_list[0][-12:-3]

'20200115a'

In [7]:
#Delete all nc files 
os.system("rm *.nc")

#Load old file
for i in tqdm(range(len(file_list))):
#for i in range(0,1):
    file = xr.open_dataset(file_list[i],decode_times=False)
    flight_acronym = file_list[i][-12:-3]
    flight_id = "HALO-"+file_list[i][-8:-4]

    #Create new netCDF file
    ncfile = netCDF4.Dataset(prefix+flight_acronym+'.nc',mode='w',format='NETCDF4_CLASSIC')

    #Create attributes
    ncfile.title='EUERC4A position and attitude data'
    ncfile.subtitle="10Hz subset based on DLR BAHAMAS data, processed by A.Giez, V.Dreiling, M.Zoeger, Ch. Mallaun; email: andreas.giez@dlr.de"
    ncfile.mission="EUREC4A"
    ncfile.platform="HALO"
    ncfile.flight_id=str(flight_id)
    ncfile.date_last_revised=file.date_last_revised
    today = date.today()
    ncfile.file_created = "File created by M. Klingebiel (email: marcus.klingebiel@uni-leipzig.de) on "+today.strftime("%B %d, %Y")
    

    #Create dimension
    dim_dim = ncfile.createDimension('time', len(file.TIME.values[:]))

    #Create variables
    time = ncfile.createVariable('time', np.float64, ('time',),fill_value=-9900.0)
    time.units = "s"
    time.long_name = 'time in seconds since 1970-1-1 (UTC)'
    

    lat = ncfile.createVariable('lat', np.float64, ('time',),fill_value=-9900.0)
    lat.units = 'degree'
    lat.long_name = 'WGS84 Datum/Latitude'

    lon = ncfile.createVariable('lon', np.float64, ('time',),fill_value=-9900.0)
    lon.units = 'degree'
    lon.long_name = 'WGS84 Datum/Longitude'

    alt = ncfile.createVariable('alt', np.float32, ('time',),fill_value=-9900.0)
    alt.units = 'm'
    alt.long_name = 'WGS84 Datum/Elliptical Height'

    roll = ncfile.createVariable('roll', np.float32, ('time',),fill_value=-9900.0)
    roll.units = 'degree'
    roll.long_name = 'roll angle'

    pitch = ncfile.createVariable('pitch', np.float32, ('time',),fill_value=-9900.0)
    pitch.units = 'degree'
    pitch.long_name = 'pitch angle'

    heading = ncfile.createVariable('heading', np.float32, ('time',),fill_value=-9900.0)
    heading.units = 'degree'
    heading.long_name = 'true heading'
    
    tas = ncfile.createVariable('tas', np.float32, ('time',),fill_value=-9900.0)
    tas.units = 'm/s'
    tas.long_name = 'True Air Speed'

    #Write variables
    time[:]    = file.TIME.values[:]
    lat[:]     = file.IRS_LAT.values[:]
    lon[:]     = file.IRS_LON.values[:]
    alt[:]     = file.IRS_ALT.values[:]
    roll[:]    = file.IRS_PHI.values[:]
    pitch[:]   = file.IRS_THE.values[:]
    heading[:] = file.IRS_HDG.values[:]
    tas[:]     = file.TAS.values[:]


    #Close file
    ncfile.close()




/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """
